# Testando o codigo extraction

In [ ]:
#Importando as bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os

In [ ]:
#Criando os intervalos de tempo para o DataFrame
START_YEAR = 1970
END_YEAR = datetime.now().year - 1

In [ ]:
# URLs base para cada tipo de tabela
URL_TEMPLATES = [
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_05",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_04"
]

In [ ]:
#Verifica a disponibilidade da url
def fetch_page_content(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.content

In [ ]:
# Função para padronizar os nomes das colunas
def standardize_column_name(column):
    column = column.strip()  # Remove extra spaces
    column = column.lower()  # Convert to lowercase
    column = column.replace(' ', '_')  # Replace spaces with underscores
    column = column.replace('(', '')  # Remove parentheses
    column = column.replace(')', '')  # Remove parentheses
    column = column.replace('.', '')  # Remove periods
    return column

In [ ]:
def parse_table_content_with_category(content):
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find('table', class_='tb_base tb_dados')
    headers = [header.text.strip() for header in table.find_all('th')]
    headers.insert(0, 'Categoria')
    rows = []

    current_category = None

    for row in table.find_all('tr'):
        cells = row.find_all('td')

        if len(cells) == 2:
            product = cells[0].text.strip()
            quantity = cells[1].text.strip().replace('.', '').replace('-', '0')
            quantity = int(quantity) if quantity.isdigit() else 0

            # Verifica se o texto está em maiúsculas e não contém números
            if product.isupper() and not any(c.isdigit() for c in product):
                # Se for uma linha de soma total, assume como a categoria atual
                current_category = product
            else:
                # Adiciona a linha ao DataFrame com a categoria atual
                rows.append([current_category, product, quantity])

    return headers, rows

In [ ]:
#Extrai a tabela para cada ano
def extract_table_data(url, year):
    try:
        content = fetch_page_content(url)
        headers, rows = parse_table_content_with_category(content)
        df = pd.DataFrame(rows, columns=headers)
        df['Ano'] = year
        df.columns = [standardize_column_name(col) for col in df.columns]
        df.fillna(0, inplace=True)
        return df
    except Exception as e:
        print(f"Erro ao extrair dados do ano {year}: {e}")
        return pd.DataFrame()

In [ ]:
#Extrai todas as tabelas dentro do intervalo de anos definido
def extract_table_all_data(url_template, start_year, end_year):
    all_data = pd.DataFrame()
    for year in range(start_year, end_year + 1):
        url = url_template.format(year=year)
        year_data = extract_table_data(url, year)
        if not year_data.empty:
            all_data = pd.concat([all_data, year_data], ignore_index=True)
            print(f"Dados do ano {year} extraídos com sucesso.")
    return all_data

In [ ]:
#Pega os headers da tabela, para ser usado como coluna no pivot_dataframe
def get_table_headers(url, end_year):
    content = fetch_page_content(url.format(year=end_year))
    headers, _ = parse_table_content_with_category(content)
    return headers

In [ ]:
#Pega o titulo da tabela e cria o nome do arquivo CSV com o datetime do dia da extração
def get_filename_from_page(end_year, url):
    content = fetch_page_content(url.format(year=end_year))
    soup = BeautifulSoup(content, 'html.parser')
    p_element = soup.find('p', {'class': 'text_center'})
    if p_element:
        filename_base = p_element.text.strip().replace(f' [{end_year}]', '').replace(' ', '_').replace(',', '')
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'{filename_base}_{timestamp}.csv'
        return filename
    return None

In [ ]:
#Função para salvar o DataFrame em um CSV
def save_to_csv(df, filename):
    df.to_csv(filename, index=False)

### Sequencia de execução das funções para gerar o DataFrame final

In [ ]:
all_data = extract_table_all_data(URL_TEMPLATES[1], START_YEAR, END_YEAR)
columns = get_table_headers(URL_TEMPLATES[1], END_YEAR)
filename = get_filename_from_page(END_YEAR, URL_TEMPLATES[1])

In [ ]:
all_data.head(15)

### Execução para salvar todos os DataFrames extraidos a partir do Web Scraping

In [ ]:
# # Função principal para extrair dados de todos os URLs
# def extract_and_save_all_data(url_templates, start_year, end_year):
#     for url_template in url_templates:
#         all_data = extract_table_all_data(url_template, start_year, end_year)
#         if not all_data.empty:
#             filename = get_filename_from_page(end_year, url_template)
#             if filename:
#                 save_to_csv(all_data, filename)
#                 print(f"Dados salvos em {filename}")
# # Executa a extração de dados
# extract_and_save_all_data(URL_TEMPLATES, START_YEAR, END_YEAR)

In [ ]:
# for url in URL_TEMPLATES:
#     filename = get_filename_from_page(END_YEAR, url)
#     print(filename)

In [ ]:
filename

In [ ]:
def process_string(input_string):
    # Remove the file extension
    without_extension = input_string.split('.')[0]
    
    # Remove the date and time part
    parts = without_extension.split('__')
    if len(parts) > 1:
        without_date_time = parts[0]
    else:
        without_date_time = parts[0].rsplit('_', 2)[0]
    
    # Replace special characters and adjust the format
    result = without_date_time.lower().replace('ç', 'c').replace('ã', 'a').replace('é', 'e').replace('í', 'i').replace('ú', 'u').replace('á', 'a').replace('ó', 'o').replace('ê', 'e').replace('ô', 'o').replace(' ', '_').replace('__', '_').replace('_e_', '_').replace('_de_', '_').replace('_', '_')
    
    return result

In [ ]:
filename = process_string(filename)
filename

In [ ]:
#Carregando as variaveis de ambiente
load_dotenv()

#Azure
# dbname=os.getenv('PGDATABASE')
# user=os.getenv('PGUSER')
# password=os.getenv('PGPASSWORD')
# host=os.getenv('PGHOST')
# port=os.getenv('PGPORT')

#Heroku
dbname=os.getenv('HDATABASE')
user=os.getenv('HUSER')
password=os.getenv('HPASSWORD')
host=os.getenv('HHOST')
port=os.getenv('HPORT')

In [ ]:
# Conectando ao banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)
cur = conn.cursor()

# Função para mapear os tipos de dados do Pandas para PostgreSQL
def mapear_tipos_pandas_para_postgresql(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_object_dtype(dtype):
        return "TEXT"
    else:
        return "TEXT"  # Tipo padrão

# Função para criar a tabela no PostgreSQL com base nos dtypes do DataFrame
def criar_tabela_com_dtypes(nome_tabela, df, schema='public'):
    nome_completo_tabela = f"{schema}.{nome_tabela}"
    colunas_com_tipos = ", ".join([f"{col} {mapear_tipos_pandas_para_postgresql(dtype)}" for col, dtype in df.dtypes.items()])
    
    # Criar a query SQL para criar a tabela
    query = f"CREATE TABLE IF NOT EXISTS {nome_completo_tabela} ({colunas_com_tipos});"
    
    # Executar a query
    cur.execute(query)
    conn.commit()

# Inserindo os dados do DataFrame no banco de dados
def inserir_dados(nome_tabela, df):
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        values = ', '.join(['%s'] * len(row))
        insert_query = sql.SQL(f"INSERT INTO {nome_tabela} ({columns}) VALUES ({values})")
        cur.execute(insert_query, tuple(row))
    conn.commit()

# Nome da tabela
nome_tabela = filename

# Criando a tabela
criar_tabela_com_dtypes(nome_tabela, all_data)

# Inserindo os dados
inserir_dados(nome_tabela, all_data)

# Fechando a conexão
cur.close()
conn.close()